In [4]:
# Importing necessary libraries
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix

In [5]:
# Loading the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [6]:
# Reshaping the data to match the input shape expected by the model
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

In [7]:
# Setting some hyperparameters
batch_size = 128
num_classes = 10
epochs = 10

In [8]:
# Converting labels to categorical (one-hot encoding)
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [9]:
# Normalizing pixel values to be between 0 and 1
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [10]:
# Creating the neural network model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compiling the model
model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])

# Adding early stopping for model training
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [11]:
# Training the model
hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test), callbacks=[early_stopping])

# Saving the trained model
model.save('mnist.h5')
print("Saving the model as mnist.h5")

Epoch 1/10
469/469 [==============================] - 182s 386ms/step - loss: 2.2677 - accuracy: 0.1962 - val_loss: 2.2255 - val_accuracy: 0.5323
Epoch 2/10
469/469 [==============================] - 170s 363ms/step - loss: 2.1956 - accuracy: 0.3575 - val_loss: 2.1314 - val_accuracy: 0.6715
Epoch 3/10
469/469 [==============================] - 169s 361ms/step - loss: 2.0953 - accuracy: 0.4748 - val_loss: 2.0019 - val_accuracy: 0.7327
Epoch 4/10
469/469 [==============================] - 169s 360ms/step - loss: 1.9551 - accuracy: 0.5609 - val_loss: 1.8196 - val_accuracy: 0.7686
Epoch 5/10
469/469 [==============================] - 172s 367ms/step - loss: 1.7662 - accuracy: 0.6187 - val_loss: 1.5811 - val_accuracy: 0.7912
Epoch 6/10
469/469 [==============================] - 172s 366ms/step - loss: 1.5389 - accuracy: 0.6583 - val_loss: 1.3154 - val_accuracy: 0.8052
Epoch 7/10
469/469 [==============================] - 175s 372ms/step - loss: 1.3158 - accuracy: 0.6879 - val_loss: 1.0742 -

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
# Evaluating the model on the test data
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.6699315905570984
Test accuracy: 0.8508999943733215


In [14]:
import numpy as np
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Classification Report
print("Classification Report:")
print(classification_report(y_true, y_pred_classes))

# Confusion Matrix
confusion_mtx = confusion_matrix(y_true, y_pred_classes)
print("Confusion Matrix:")
print(confusion_mtx)

313/313 [==============================] - 14s 46ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.95      0.92       980
           1       0.88      0.96      0.91      1135
           2       0.89      0.85      0.87      1032
           3       0.80      0.89      0.84      1010
           4       0.83      0.80      0.82       982
           5       0.92      0.64      0.76       892
           6       0.86      0.91      0.89       958
           7       0.91      0.85      0.88      1028
           8       0.80      0.79      0.80       974
           9       0.76      0.84      0.80      1009

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000

Confusion Matrix:
[[ 934    0    1    3    0    2   21    1   18    0]
 [   0 1087   19   10    1    1    5    0    9    3]
 [  17    9  873   13   17    1   33 